#FastBioDL

In [1]:
!git clone https://github.com/swargo98/Parallel-Fastq-Downloader.git

Cloning into 'Parallel-Fastq-Downloader'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 159 (delta 56), reused 125 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (159/159), 2.03 MiB | 14.22 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [2]:
!pip install -r /content/Parallel-Fastq-Downloader/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.7/157.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:

In [1]:
!python /content/Parallel-Fastq-Downloader/fastbiodl.py -i /content/Parallel-Fastq-Downloader/accessions.txt

1754372950.382994 -- INFO: Fetching URLs for SRR15852393 from NCBI SRA using field 'sra_ftp'
1754372951.017752 -- INFO: Fetching URLs for SRR15852394 from NCBI SRA using field 'sra_ftp'
1754372952.015931 -- INFO: Fetching URLs for SRR15852396 from NCBI SRA using field 'sra_ftp'
1754372952.874861 -- INFO: Fetching URLs for SRR15852397 from NCBI SRA using field 'sra_ftp'
1754372954.782608 -- INFO: Fetching URLs for SRR15852398 from NCBI SRA using field 'sra_ftp'
1754372956.293794 -- INFO: Fetching URLs for SRR15852400 from NCBI SRA using field 'sra_ftp'
1754372958.971998 -- INFO: Fetching URLs for SRR15852401 from NCBI SRA using field 'sra_ftp'
1754372960.134597 -- INFO: Fetching URLs for SRR15852402 from NCBI SRA using field 'sra_ftp'
1754372960.897393 -- INFO: Fetching URLs for SRR15852403 from NCBI SRA using field 'sra_ftp'
1754372962.810382 -- INFO: Fetching URLs for SRR15852404 from NCBI SRA using field 'sra_ftp'
1754372966.528210 -- INFO: Total files to download: 10: [('https://sra

#Prefetch

In [2]:
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.1.0/sratoolkit.3.1.0-ubuntu64.tar.gz
!tar -xzf sratoolkit.3.1.0-ubuntu64.tar.gz

--2025-08-05 05:52:35--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.1.0/sratoolkit.3.1.0-ubuntu64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.31, 130.14.250.7, 130.14.250.10, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93495998 (89M) [application/x-gzip]
Saving to: ‘sratoolkit.3.1.0-ubuntu64.tar.gz’

sratoolkit.3.1.0-ub 100%[===================>]  89.16M  58.5MB/s    in 1.5s    

2025-08-05 05:52:37 (58.5 MB/s) - ‘sratoolkit.3.1.0-ubuntu64.tar.gz’ saved [93495998/93495998]



In [5]:
import os, pathlib, sys
toolkit_dir = pathlib.Path().absolute() / "sratoolkit.3.1.0-ubuntu64" / "bin"
os.environ["PATH"] = f"{toolkit_dir}:{os.environ['PATH']}"

# now every new `!` cell can find prefetch
!prefetch --version


prefetch : 3.1.0



In [6]:
import subprocess
import time
import os
import sys
import argparse

def get_directory_size(path):
    """Calculates the size of a directory in bytes."""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def run_prefetch_with_logging(accession_list_file, output_dir):
    """
    Runs prefetch for each accession in the list and logs second-by-second throughput.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        with open(accession_list_file, 'r') as f:
            accessions = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"Error: Accession list file not found at {accession_list_file}")
        sys.exit(1)

    print(f"Starting prefetch for {len(accessions)} accessions...")

    log_file = f'prefetch_throughput_{int(time.time())}.csv'
    with open(log_file, 'w') as lg:
        lg.write("Timestamp,Elapsed_Time(s),Current_Throughput(Mbps),Total_Downloaded(bytes)\n")

    total_downloaded_bytes = 0
    previous_total_size = 0
    start_time = time.time()

    for accession in accessions:
        print(f"Processing accession: {accession}")
        # Construct the prefetch command.
        # Assuming prefetch is in your PATH. You might need to provide the full path.
        # Adjust the command and arguments as needed for your specific use case (e.g., output directory).
        command = ["prefetch", "--eliminate-quals", accession, "-O", output_dir]

        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Monitor the download progress
        accession_output_dir = os.path.join(output_dir, accession)
        while process.poll() is None:
            time.sleep(1) # Log every second
            current_total_size = get_directory_size(accession_output_dir)
            downloaded_this_second = current_total_size - previous_total_size
            total_downloaded_bytes += downloaded_this_second
            previous_total_size = current_total_size

            elapsed_time = time.time() - start_time
            current_throughput_mbps = (downloaded_this_second * 8) / (1024 * 1024) # Convert bytes to Mbps

            timestamp = time.time()
            with open(log_file, 'a') as lg:
                lg.write(f"{timestamp},{elapsed_time:.2f},{current_throughput_mbps:.2f},{total_downloaded_bytes}\n")

            print(f"  {elapsed_time:.2f}s - Downloaded: {current_total_size} bytes, Throughput: {current_throughput_mbps:.2f} Mbps")


        # Check for errors after the process finishes
        stdout, stderr = process.communicate()
        if process.returncode != 0:
            print(f"Error running prefetch for {accession}:")
            print(stderr.decode())
        else:
            print(f"Successfully processed accession: {accession}")
            # remove downloaded file
            import shutil
            shutil.rmtree(accession_output_dir)

    end_time = time.time()
    print(f"\nFinished prefetch for all accessions in {end_time - start_time:.2f} seconds.")
    print(f"Throughput data logged to {log_file}")

In [7]:
run_prefetch_with_logging('/content/Parallel-Fastq-Downloader/accessions.txt', '/content/outdir')

Starting prefetch for 10 accessions...
Processing accession: SRR15852393
  1.00s - Downloaded: 31538528 bytes, Throughput: 240.62 Mbps
  2.01s - Downloaded: 117812576 bytes, Throughput: 658.22 Mbps
  3.01s - Downloaded: 184642912 bytes, Throughput: 509.88 Mbps
  4.01s - Downloaded: 276545536 bytes, Throughput: 701.16 Mbps
  5.01s - Downloaded: 384843104 bytes, Throughput: 826.24 Mbps
  6.01s - Downloaded: 488320352 bytes, Throughput: 789.47 Mbps
  7.01s - Downloaded: 596573536 bytes, Throughput: 825.91 Mbps
  8.01s - Downloaded: 693063008 bytes, Throughput: 736.16 Mbps
  9.02s - Downloaded: 742088032 bytes, Throughput: 374.03 Mbps
  10.02s - Downloaded: 817635328 bytes, Throughput: 576.38 Mbps
  11.02s - Downloaded: 918548480 bytes, Throughput: 769.91 Mbps
  12.02s - Downloaded: 1019043168 bytes, Throughput: 766.71 Mbps
  13.02s - Downloaded: 1115102560 bytes, Throughput: 732.88 Mbps
  14.02s - Downloaded: 1195535712 bytes, Throughput: 613.66 Mbps
  15.02s - Downloaded: 1213537632 byte

In [8]:
!rm -rf /content/outdir

#pysradb

In [9]:
!pip install pysradb

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.0/171.0 kB 4.4 MB/s eta 0:00:00
  Created wheel for requests-ftp: filename=requests_ftp-0.3.1-py3-none-any.whl size=8166 sha256=83f76be5869c96e04031dce5662bcdc997784f38840b969279f9844080cda454
  Stored in directory: /root/.cache/pip/wheels/c9/06/7b/2ce24ee089ed6028ebf8073740d9c35b819de74cf66ccc3077
Successfully built requests-ftp


In [10]:
import subprocess
import time
import os
import sys
import argparse
import datetime

def get_directory_size(path):
    """Calculates the size of a directory in bytes."""
    total_size = 0
    if not os.path.exists(path):
        return 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # Avoid issues with symbolic links pointing outside the directory
            if not os.path.islink(fp):
                try:
                    total_size += os.path.getsize(fp)
                except OSError:
                    # Handle potential errors like permission denied or file disappearing
                    pass
    return total_size

def run_pysradb_download_with_logging(accession_list_file, output_dir):
    """
    Runs pysradb download for each accession in the list and logs second-by-second throughput.
    """
    # Create the main output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        with open(accession_list_file, 'r') as f:
            accessions = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"Error: Accession list file not found at {accession_list_file}")
        sys.exit(1)

    print(f"Starting pysradb download for {len(accessions)} accessions...")

    # Create a unique log file name based on timestamp
    log_file = f'pysradb_download_throughput_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    with open(log_file, 'w') as lg:
        lg.write("Timestamp,Elapsed_Time(s),Current_Throughput(Mbps),Total_Downloaded(bytes),Accession\n")

    total_downloaded_bytes_overall = 0
    start_time_overall = time.time()

    for accession in accessions:
        print(f"Processing accession: {accession}")

        # pysradb download saves files within the specified output directory.
        # The exact subdirectory structure might vary, but monitoring the output_dir itself
        # or a subdirectory created by pysradb is necessary.
        # Based on pysradb usage, it often downloads to the specified --outdir.
        # We will monitor the size of the output_dir.

        # Construct the pysradb download command.
        # Assuming pysradb is in your PATH. You might need to provide the full path.
        # Using --outdir to specify the download location.
        command = ["pysradb", "download", "-y", "-t", "8", "--out-dir", output_dir, "-p", accession]

        process = None # Initialize process to None
        try:
            # Initiate the pysradb download process
            process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) # Use text=True for string output

            # Monitor the download progress by checking the size of the output directory
            print(f"Monitoring output directory: {output_dir}")
            previous_total_size = get_directory_size(output_dir)
            start_time_accession = time.time() # Timer for this specific accession

            # Give pysradb a moment to start the download and potentially create files
            time.sleep(2)

            # Monitor the process while it's running
            while process.poll() is None:
                time.sleep(1) # Log approximately every second

                # Calculate the current total size of the output directory
                current_total_size = get_directory_size(output_dir)

                # Bytes downloaded in the last second interval
                downloaded_this_second = current_total_size - previous_total_size
                total_downloaded_bytes_overall += downloaded_this_second # Accumulate to overall total
                previous_total_size = current_total_size # Update previous size for the next interval

                elapsed_time_overall = time.time() - start_time_overall
                # Convert bytes per second to Mbps (Mega bits per second)
                current_throughput_mbps = (downloaded_this_second * 8) / (1024 * 1024)

                timestamp = time.time()
                # Log the data point
                with open(log_file, 'a') as lg:
                    lg.write(f"{timestamp},{elapsed_time_overall:.2f},{current_throughput_mbps:.2f},{total_downloaded_bytes_overall},{accession}\n")

                # Print progress update to console
                print(f"  {elapsed_time_overall:.2f}s - Accession {accession} - Downloaded this second: {downloaded_this_second} bytes, Current Throughput: {current_throughput_mbps:.2f} Mbps")

            # --- End of monitoring loop ---

            # Process has finished, get the final output and check return code
            stdout, stderr = process.communicate()

            if process.returncode != 0:
                # Error occurred during pysradb download
                print(f"Error running pysradb download for {accession} (exit code {process.returncode}):")
                print(stderr)
            else:
                # pysradb download completed successfully
                final_total_size = get_directory_size(output_dir)
                # Account for any bytes downloaded in the final interval after the loop exited
                downloaded_in_final_interval = final_total_size - previous_total_size
                total_downloaded_bytes_overall += downloaded_in_final_interval

                elapsed_time_overall = time.time() - start_time_overall
                 # Convert bytes per second to Mbps for the final interval
                current_throughput_mbps = (downloaded_in_final_interval * 8) / (1024 * 1024)
                timestamp = time.time()
                with open(log_file, 'a') as lg:
                     lg.write(f"{timestamp},{elapsed_time_overall:.2f},{current_throughput_mbps:.2f},{total_downloaded_bytes_overall},{accession}\n")
                print(f"  {elapsed_time_overall:.2f}s - Accession {accession} - Downloaded in final interval: {downloaded_in_final_interval} bytes, Current Throughput: {current_throughput_mbps:.2f} Mbps")


                print(f"Successfully processed accession: {accession} in {time.time() - start_time_accession:.2f} seconds.")

        except FileNotFoundError:
             print(f"Error: 'pysradb' command not found. Is pysradb installed and in your PATH?")
             sys.exit(1)
        except Exception as e:
            # Catch other potential errors during subprocess execution
            print(f"An unexpected error occurred while processing accession {accession}: {e}")
            if process and process.poll() is None:
                 print(f"Attempting to terminate process {process.pid}")
                 process.terminate()
                 try:
                     process.wait(timeout=5)
                 except subprocess.TimeoutExpired:
                     print(f"Process {process.pid} did not terminate, killing.")
                     process.kill()


    end_time_overall = time.time()
    print(f"\nFinished pysradb download for all accessions in {end_time_overall - start_time_overall:.2f} seconds.")
    print(f"Total downloaded bytes (estimated by directory size): {total_downloaded_bytes_overall}")
    print(f"Throughput data logged to {log_file}")

In [12]:
run_pysradb_download_with_logging('/content/Parallel-Fastq-Downloader/accessions.txt', '/content/outdir')

Starting pysradb download for 10 accessions...
Processing accession: SRR15852393
Monitoring output directory: /content/outdir
  3.00s - Accession SRR15852393 - Downloaded this second: 0 bytes, Current Throughput: 0.00 Mbps
  4.00s - Accession SRR15852393 - Downloaded this second: 0 bytes, Current Throughput: 0.00 Mbps
  5.00s - Accession SRR15852393 - Downloaded this second: 0 bytes, Current Throughput: 0.00 Mbps
  6.00s - Accession SRR15852393 - Downloaded this second: 0 bytes, Current Throughput: 0.00 Mbps
  7.00s - Accession SRR15852393 - Downloaded this second: 0 bytes, Current Throughput: 0.00 Mbps
  8.00s - Accession SRR15852393 - Downloaded this second: 58552320 bytes, Current Throughput: 446.72 Mbps
  9.01s - Accession SRR15852393 - Downloaded this second: 122851328 bytes, Current Throughput: 937.28 Mbps
  10.01s - Accession SRR15852393 - Downloaded this second: 145084416 bytes, Current Throughput: 1106.91 Mbps
  11.01s - Accession SRR15852393 - Downloaded this second: 75485184

In [13]:
!rm -rf /content/outdir